In [ ]:
import os
import duckdb
import pandas as pd
from typing import List

def criar_diretorio(caminho: str) -> None:
    """Garante que o diretório existe."""
    os.makedirs(caminho, exist_ok=True)

def salvar_dataframe_em_parquet_particionado_duckdb(
    df: pd.DataFrame,
    caminho_destino: str,
    colunas_particao: List[str]
) -> None:
    """
    Salva um DataFrame como Parquet particionado usando DuckDB,
    criando as pastas no formato hive (coluna=valor/).
    """
    criar_diretorio(caminho_destino)

    con = duckdb.connect(database=':memory:')  # Pode ser in-memory porque só queremos salvar

    con.register('temp_table', df)

    # Cria diretório de destino absoluto
    caminho_destino = os.path.abspath(caminho_destino)

    # COPY TO ... (partition_by colunas)
    con.execute(f"""
        COPY temp_table 
        TO '{caminho_destino}' 
        (FORMAT PARQUET, PARTITION_BY ({', '.join(colunas_particao)}))
    """)

    con.close()



df_pessoas = gerar_base_historica_ficticia()

salvar_dataframe_em_parquet_particionado_duckdb(
    df=df_pessoas,
    caminho_destino="../../dados/ficticios/uf-municipio",
    colunas_particao=['uf', 'municipio']
)

In [ ]:
from faker import Faker
import numpy as np

def gerar_base_historica_ficticia():

    # Inicializa o gerador de dados falsos
    faker = Faker('pt_BR')
    
    # Define o número de linhas que você quer gerar
    n_pessoas = 1000
    
    # Gera os dados de pessoas com municípios
    dados_pessoas = {
        'uf': np.random.choice(['SP', 'RJ', 'MG', 'RS', 'BA'], size=n_pessoas),
        'municipio': [faker.city() for _ in range(n_pessoas)],
        'nome': [faker.name() for _ in range(n_pessoas)],
        'data_inicio': [faker.date_between(start_date='-5y', end_date='today') for _ in range(n_pessoas)],
        'data_fim': [faker.date_between(start_date='today', end_date='+5y') for _ in range(n_pessoas)],
    }
    
    # Cria o DataFrame de pessoas
    df_pessoas = pd.DataFrame(dados_pessoas)
    
    # Converte para datetime
    df_pessoas['data_inicio'] = pd.to_datetime(df_pessoas['data_inicio'])
    df_pessoas['data_fim'] = pd.to_datetime(df_pessoas['data_fim'])
    
    # Garante que data_inicio <= data_fim
    df_pessoas.loc[df_pessoas['data_inicio'] > df_pessoas['data_fim'], ['data_inicio', 'data_fim']] = \
        df_pessoas.loc[df_pessoas['data_inicio'] > df_pessoas['data_fim'], ['data_fim', 'data_inicio']].values

    return df_pessoas

    